# Smartphones and Watches advisor
In this serie of notebooks we will:
- Load text from a Youtube video playlist
- Split loaded text into appropriate chunks
- **Create a vector store to load chunk embeddings**
- Create a Question Answering chatbot:

    - with different prompts techniques
    - with different similarity measures
    - with or without chat memory 

!['Data Connection'](../../images/les_numeriques_youtube/data_connection.png)


# Create a vector store to load chunk embeddings

We will do the following steps:
- Retrieve the chunks by metadata
- Apply a **compression** method to increase chunk quality

In [1]:
# Imports 
# Env var
import sys
from dotenv import load_dotenv, find_dotenv

# Langchain
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma, MyScale, MyScaleSettings
from langchain.embeddings.openai import OpenAIEmbeddings

# SelfQuery Retriever
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

# Compressor
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# Utils
from utils import count_lines_in_myscale

In [2]:
# Env variable
sys.path.append('../..')
load_dotenv(find_dotenv())

True

In [3]:
# Load vector db
persist_directory = 'docs/chroma/'

embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

print(vectordb._collection.count())

815


In [4]:
question = """Quelles sont les caractéristiques de l'iphone 14 ?"""

In [5]:
res = vectordb.similarity_search(
    question,
    k=10
)

res[:5]

[Document(page_content="Comme prévu lors de sa canote de rentrée, Apple a annoncé non pas un, non pas deux, non pas trois, mais bien quatre iPhones. Pas d'iPhone mini au programme, mais ça c'était prévu. A la place on se retrouve avec un iPhone 14, un iPhone 14+, un iPhone 14 Pro et un iPhone 14 Pro Max et je vous propose qu'on décortique tout ça maintenant. On va commencer avec les deux entrées de gamme qui sont les iPhone 14 et les iPhone 14+.", metadata={'source': "docs/youtube/iPhone 14 ： la gamme Pro gâtée, l'autre oubliée.m4a", 'chunk': 0}),
 Document(page_content="Alors esthétiquement, il n'y a rien de nouveau par rapport aux iPhone 13 mini et aux iPhone 13 de l'année dernière. A l'intérieur, c'est quasiment la même chose, sauf que l'on retrouve la puce Apple A15 Bionic que l'on retrouvait dans les iPhone 13 Pro et 13 Pro Max en 2021. Du côté de l'iPhone 14, on garde le même gabarit qu'avec l'iPhone 13 et du côté de l'iPhone 14+, on prend le même gabarit qu'un iPhone 14 Pro Max.

In [6]:
print(("\n").join([doc.metadata["source"] for doc in res[:5]]))

docs/youtube/iPhone 14 ： la gamme Pro gâtée, l'autre oubliée.m4a
docs/youtube/iPhone 14 ： la gamme Pro gâtée, l'autre oubliée.m4a
docs/youtube/iPhone 14 ： la gamme Pro gâtée, l'autre oubliée.m4a
docs/youtube/iPhone 14 ： la gamme Pro gâtée, l'autre oubliée.m4a
docs/youtube/iPhone 13 ： dates, rumeurs, ragots....m4a


Let's look at the video title of the retrieved chunks


### Self querying retriever

A self-querying retriever is one that, as the name suggests, has the ability to query itself. Specifically, given any natural language query, the retriever uses a query-constructing LLM chain to write a structured query and then applies that structured query to it's underlying VectorStore. 

This allows the retriever to not only use the user-input query for semantic similarity comparison with the contents of stored documented, but to **also extract filters from the user query on the metadata of stored documents** and to execute those filters.


![Self Querying.png](../../images/les_numeriques_youtube/self_querying.png)

In [7]:
# Metadata informations
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The title the chunk is from. The title starts with the product name",
        type="string"
    ),
    AttributeInfo(
        name="chunk",
        description="The chunk number",
        type="integer"
    ),
]

### Self-Querying avec Chroma

In [8]:
document_content_description = "Technology product information"
llm = OpenAI(temperature=0)

retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [9]:
question = """Quelles sont les caractéristiques de l'iphone 14 ?"""

In [10]:
# A filter is applied on the name of the Youtube Video to retrieve only chunks corresponding to iPhone 14 videos
res = retriever.get_relevant_documents(question)
res

/Users/florian.bastin/Documents/Code/Octo/GenAI/langchain_discovery/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='iphone 14' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='source', value='iphone 14') limit=None


[]

[Liste des comparateurs possible de Comparator]('https://api.python.langchain.com/en/latest/chains/langchain.chains.query_constructor.ir.Comparator.html')

In [11]:
[chunk.metadata['source'] for chunk in res]

[]

### Self-Querying avec MyScale

In [12]:
# Instantiate MyScale DB (which is an online vector DB)
myscale = MyScale(embedding=embedding, config=MyScaleSettings(_env_file=find_dotenv()))

print(f"Number of lines in MyScale: {count_lines_in_myscale(myscale)}")

Number of lines in MyScale: 815


In [13]:
myscale_self_query_retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    document_contents=document_content_description,
    metadata_field_info=metadata_field_info,
    vectorstore=myscale,
    verbose=True
)

In [14]:
myscale_res = myscale_self_query_retriever.get_relevant_documents(question)

/Users/florian.bastin/Documents/Code/Octo/GenAI/langchain_discovery/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='iphone 14' filter=Comparison(comparator=<Comparator.LIKE: 'like'>, attribute='source', value='iphone 14') limit=None
query='iphone 14' filter=Comparison(comparator=<Comparator.LIKE: 'like'>, attribute='source', value='iphone 14') limit=None


In [15]:
myscale_res

[Document(page_content="Comme prévu lors de sa canote de rentrée, Apple a annoncé non pas un, non pas deux, non pas trois, mais bien quatre iPhones. Pas d'iPhone mini au programme, mais ça c'était prévu. A la place on se retrouve avec un iPhone 14, un iPhone 14+, un iPhone 14 Pro et un iPhone 14 Pro Max et je vous propose qu'on décortique tout ça maintenant. On va commencer avec les deux entrées de gamme qui sont les iPhone 14 et les iPhone 14+.", metadata={'chunk': 0, 'source': "docs/youtube/iPhone 14 ： la gamme Pro gâtée, l'autre oubliée.m4a"}),
 Document(page_content="Alors esthétiquement, il n'y a rien de nouveau par rapport aux iPhone 13 mini et aux iPhone 13 de l'année dernière. A l'intérieur, c'est quasiment la même chose, sauf que l'on retrouve la puce Apple A15 Bionic que l'on retrouvait dans les iPhone 13 Pro et 13 Pro Max en 2021. Du côté de l'iPhone 14, on garde le même gabarit qu'avec l'iPhone 13 et du côté de l'iPhone 14+, on prend le même gabarit qu'un iPhone 14 Pro Max.

In [16]:
[chunk.metadata["source"] for chunk in myscale_res]

["docs/youtube/iPhone 14 ： la gamme Pro gâtée, l'autre oubliée.m4a",
 "docs/youtube/iPhone 14 ： la gamme Pro gâtée, l'autre oubliée.m4a",
 'docs/youtube/[TEST] iPhone 14 Pro et Pro Max ： la maturité avant les changements ？.m4a',
 "docs/youtube/iPhone 14 ： la gamme Pro gâtée, l'autre oubliée.m4a"]

## Contextual Compression
[LangChain doc](https://python.langchain.com/docs/modules/data_connection/retrievers/how_to/contextual_compression/) 

One challenge with retrieval is that usually you don't know the specific queries your document storage system will face when you ingest data into the system. This means that the information most relevant to a query may be buried in a document with a lot of irrelevant text. Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this. 

The idea is simple: instead of immediately returning retrieved documents as-is, you can compress them using the context of the given query, so that only the relevant information is returned. 

“Compressing” here refers to both compressing the contents of an individual document and filtering out documents wholesale.

To use the Contextual Compression Retriever, you'll need:

- A base Retriever
- A Document Compressor

The Contextual Compression Retriever passes queries to the base Retriever, takes the initial documents and passes them through the Document Compressor. The Document Compressor takes a list of Documents and shortens it by reducing the contents of Documents or dropping Documents altogether.

!['Compression 2'](../../images/les_numeriques_youtube/compression2.png)

In [17]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [18]:
llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [19]:
question = """Quelles sont les caractéristiques de l'iPhone 14 ?"""

compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

"Apple a annoncé non pas un, non pas deux, non pas trois, mais bien quatre iPhones. Pas d'iPhone mini au programme, mais ça c'était prévu. A la place on se retrouve avec un iPhone 14, un iPhone 14+, un iPhone 14 Pro et un iPhone 14 Pro Max"
----------------------------------------------------------------------------------------------------
Document 2:

puce Apple A15 Bionic, même gabarit qu'avec l'iPhone 13, même gabarit qu'un iPhone 14 Pro Max
----------------------------------------------------------------------------------------------------
Document 3:

"un écran de 6,7 pouces", "batterie a été améliorée", "capteurs qui étaient présents sur les iPhone 13 Pro pour le principal et pour l'ultra grand-angle", "module frontal a été entièrement revu".
----------------------------------------------------------------------------------------------------
Document 4:

"On n'a pas de nouveau design sur l'écran, ce qui fait qu'on garde encore l'encoche"
